# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298543622296                   -0.85    5.1         
  2   -8.300211160088       -2.78       -1.26    1.0   90.9ms
  3   -8.300439737066       -3.64       -1.89    2.8    115ms
  4   -8.300461394051       -4.66       -2.76    1.8   97.6ms
  5   -8.300463939219       -5.59       -3.03    7.8    288ms
  6   -8.300464301226       -6.44       -3.23    2.0   96.4ms
  7   -8.300464472102       -6.77       -3.37    1.6   90.6ms
  8   -8.300464568672       -7.02       -3.52    1.2   84.1ms
  9   -8.300464621319       -7.28       -3.70    2.4    122ms
 10   -8.300464637239       -7.80       -3.88    1.5    129ms
 11   -8.300464643879       -8.18       -4.40    2.1    114ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64153558283                   -0.70    6.0         
  2   -16.67680478955       -1.45       -1.14    1.4    202ms
  3   -16.67919309911       -2.62       -1.88    3.0    242ms
  4   -16.67927674081       -4.08       -2.74    4.2    259ms
  5   -16.67928563777       -5.05       -3.09    5.1    317ms
  6   -16.67928618666       -6.26       -3.48    2.2    208ms
  7   -16.67928621474       -7.55       -3.96    2.0    211ms
  8   -16.67928622131       -8.18       -4.54    2.4    223ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32409921072                   -0.56    6.6         
  2   -33.33254244605       -2.07       -1.00    1.1    649ms
  3   -33.33402278784       -2.83       -1.74    4.0    803ms
  4   -33.33427041808       -3.61       -2.63    3.5    764ms
  5   -33.33694093777       -2.57       -2.52   11.8    1.26s
  6   -33.33694136018       -6.37       -2.50    2.2    677ms
  7   -33.33694328199       -5.72       -2.91    1.0    588ms
  8   -33.33694355956       -6.56       -3.27    3.9    695ms
  9   -33.33694373027       -6.77       -3.65    3.2    726ms
 10   -33.33694377496       -7.35       -4.09    2.2    691ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298337007871                   -0.85    5.2         
  2   -8.300242193236       -2.72       -1.59    1.0   74.8ms
  3   -8.300430961864       -3.72       -2.56    3.1   87.2ms
  4   -8.300387996997   +   -4.37       -2.31    4.0    113ms
  5   -8.300464078577       -4.12       -3.41    1.0   67.3ms
  6   -8.300464571681       -6.31       -3.75    9.9    195ms
  7   -8.300464637500       -7.18       -4.27    2.0   90.6ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32518677314                   -0.56    7.2         
  2   -33.31354661743   +   -1.93       -1.27    1.0    556ms
  3   -14.79906908392   +    1.27       -0.43    6.0    1.18s
  4   -33.28683191271        1.27       -1.57    5.8    1.11s
  5   -33.20825749610   +   -1.10       -1.31    3.4    899ms
  6   -31.09983893970   +    0.32       -0.89    4.8    934ms
  7   -33.33013617948        0.35       -2.12    4.9    923ms
  8   -33.33552407022       -2.27       -2.21    2.6    843ms
  9   -33.33507059983   +   -3.34       -2.39    2.1    694ms
 10   -33.33676009002       -2.77       -2.76    1.8    625ms
 11   -33.33685292859       -4.03       -3.00    2.9    747ms
 12   -33.33692225913       -4.16      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.